In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

import keras
from keras import backend as K
from keras import metrics
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.engine import Layer, InputSpec
from keras.utils.generic_utils import get_custom_objects
from keras.engine.topology import Input
from keras.engine.training import Model
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.core import Activation, SpatialDropout2D
from keras.layers.merge import concatenate, Add
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

path_main = '/kaggle/input/Kannada-MNIST'
path_train = os.path.join(path_main, 'train.csv')
path_test = os.path.join(path_main, 'test.csv')
path_sample_sub = os.path.join(path_main, 'sample_submission.csv')
path_dig_mnist = os.path.join(path_main, 'Dig-MNIST.csv')

In [ ]:
version = 0.0
basic_name = f'Kannada_Minst_v{version}'
save_model_name = basic_name + '.model'

In [ ]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

In [ ]:
# Load the data
train_df = pd.read_csv(path_train)
X_train = np.array((train_df.iloc[:,1:].values.astype('float32'))/255).reshape(-1,28,28,1) # all pixel values
y_train = np.array(train_df.iloc[:,0].values.astype('int32')) # only labels i.e targets digits
y_train_ohe = one_hot(y_train, 10)
del train_df

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train_ohe, test_size = 0.2, stratify=y_train_ohe, random_state=28)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.10,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
def BatchActivate(x):
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = Conv2D(filters, size, strides=strides, padding=padding)(x)
    if activation == True:
        x = BatchActivate(x)
    return x

def inception_block(blockInput, filters):
    branchOne = Conv2D(filters, (1,1), activation='relu', padding='same')(blockInput)
    
    branchTwo = Conv2D(int(filters/2), (1,1), activation='relu', padding='same')(blockInput)
    branchTwo = Conv2D(filters, (3,3), activation='relu', padding='same')(branchTwo)
    
    branchThree = Conv2D(int(filters/2), (1,1), activation='relu', padding='same')(blockInput)
    branchThree = Conv2D(filters, (5,5), activation='relu', padding='same')(branchThree)
    
    branchFour = MaxPooling2D((3,3), strides=(1,1), padding='same')(blockInput)
    branchFour = Conv2D(filters, (1,1), activation='relu', padding='same')(branchFour)
    
    x = Add()([branchOne, branchTwo, branchThree, branchFour])
    x = BatchActivate(x)
    return x

def inception_block_small(blockInput, filters):
    branchOne = Conv2D(filters, (1,1), activation='relu', padding='same')(blockInput)
    
    branchTwo = Conv2D(int(filters/2), (1,1), activation='relu', padding='same')(blockInput)
    branchTwo = Conv2D(filters, (2,2), activation='relu', padding='same')(branchTwo)
    
    branchThree = Conv2D(int(filters/2), (1,1), activation='relu', padding='same')(blockInput)
    branchThree = Conv2D(filters, (3,3), activation='relu', padding='same')(branchThree)
    
    branchFour = MaxPooling2D((2,2), strides=(1,1), padding='same')(blockInput)
    branchFour = Conv2D(filters, (1,1), activation='relu', padding='same')(branchFour)
    
    x = Add()([branchOne, branchTwo, branchThree, branchFour])
    x = BatchActivate(x)
    return x


def residual_block(blockInput, num_filters=16, batch_activate = False):
    x = BatchActivate(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = Add()([x, blockInput])
    if batch_activate:
        x = BatchActivate(x)
    return x

In [ ]:
def build_model(input_shape, start_filters, start_neurons, dropoutratio = 0.25):
    
    input_layer = Input(input_shape)
    conv1 = Conv2D(start_filters * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = inception_block(conv1,start_filters * 2)
    conv1 = residual_block(conv1,start_filters * 2, True)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(dropoutratio)(pool1)

    conv2 = Conv2D(start_filters * 2, (3,3), activation=None, padding="same")(pool1)
    conv2 = inception_block_small(conv2,start_filters * 4)
    conv2 = residual_block(conv2,start_filters * 4, True)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(dropoutratio)(pool2)

    flat1 = Flatten()(pool2)
    dense1 = Dense(start_neurons, activation = "relu")(flat1)
    dout1 = Dropout(dropoutratio)(dense1)
    output_layer = Dense(10, activation = "softmax")(dout1)
    model = Model(inputs=input_layer, outputs=output_layer) 
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[keras.metrics.Precision(), keras.metrics.Recall()])
    
    return model

In [ ]:
epochs = 75
batch_size = 32

model = build_model(input_shape=((28, 28, 1)), start_filters=32, start_neurons=128, dropoutratio=0.25)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', mode = 'min',patience=10, verbose=1)
model_checkpoint = ModelCheckpoint(save_model_name, monitor='val_loss', mode = 'min', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode = 'min',factor=0.5, patience=3, min_lr=0.0001, verbose=1)

history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), validation_data=(X_valid, y_valid), epochs=epochs,
                    steps_per_epoch=len(X_train) / batch_size, callbacks=[early_stopping, model_checkpoint, reduce_lr], verbose=True)

In [ ]:
import matplotlib.pyplot as plt

# Look at confusion matrix 

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
y_pred = model.predict(X_valid)
# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred,axis = 1) 
# Convert validation observations to one hot vectors
y_true = np.argmax(y_valid,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
test_df = pd.read_csv(path_test)
submission = pd.read_csv(path_sample_sub)
X_test = np.array((test_df.iloc[:,1:].values.astype('float32'))/255).reshape(-1,28,28,1) # all pixel values

In [ ]:
# predict results
results = model.predict(X_test)

# select the indix with the maximum probability
results = np.argmax(results, axis = 1)

In [ ]:
submission['label'] = results
submission.to_csv("submission.csv", index=False)